In [2]:
import numpy as np
import cv2
from object_detection.network.edgetpumodel import EdgeTPUModel
from object_detection.network.utils import plot_one_box, Colors, get_image_tensor, xyxy2xywh

In [ ]:
import cv2
img = cv2.imread('/home/pi/test_depth/image0012.png')
cv2.imshow('',img)